#PyTorch 2 Quick Intro
  * Reference notebook - https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/pytorch_2_intro.ipynb
  * Reference book chapter - https://www.learnpytorch.io/pytorch_2_intro/
  * PyTorch 2.0 release notes - https://pytorch.org/blog/pytorch-2.0-release/

In [2]:
import torch
print(torch.__version__)

2.5.1+cu121


In [3]:
!nvidia-smi

Mon Dec  2 13:39:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

##Before PyTorch 2.0

In [4]:
import torch
import torchvision

model = torchvision.models.resnet50()

##After PyTorch 2.0

In [5]:
import torch
import torchvision

model = torchvision.models.resnet50() # note: this could any model
compiled_model = torch.compile(model)

### Training code

### Testing code

#0. Getting setup

In [6]:
import torch

# Check PyTorch version
pt_version = torch.__version__
print(f"[INFO] Current PyTorch version: {pt_version} (should be 2.x+)")

# Install PyTorch 2.0 if necessary
if pt_version.split(".")[0] == "1": # Check if PyTorch version begins with 1
    !pip3 install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
    print("[INFO] PyTorch 2.x installed, if you're on Google Colab, you may need to restart your runtime.\
          Though as of April 2023, Google Colab comes with PyTorch 2.0 pre-installed.")
    import torch
    pt_version = torch.__version__
    print(f"[INFO] Current PyTorch version: {pt_version} (should be 2.x+)")
else:
    print("[INFO] PyTorch 2.x installed, you'll be able to use the new features.")

[INFO] Current PyTorch version: 2.5.1+cu121 (should be 2.x+)
[INFO] PyTorch 2.x installed, you'll be able to use the new features.


##1. รับข้อมูล GPU
เหตุใดจึงต้องรับข้อมูล GPU

เนื่องจากคุณสมบัติของ PyTorch 2.0 (torch.compile()) ทำงานได้ดีที่สุดบน NVIDIA GPU รุ่นใหม่

NVIDIA GPU รุ่นใหม่คืออะไร?

หากต้องการทราบว่า GPU ของคุณเข้ากันได้หรือไม่ โปรดดูคะแนนความเข้ากันได้ของ NVIDIA GPU - https://developer.nvidia.com/cuda-gpus

หาก GPU ของคุณมีคะแนน 8.0+ ก็สามารถใช้ประโยชน์จากฟีเจอร์ PyTorch 2.0 ใหม่ได้เกือบทั้งหมด

GPU ที่ต่ำกว่า 8.0 ยังคงสามารถใช้ประโยชน์จาก PyTorch 2.0 ได้ อย่างไรก็ตาม การปรับปรุงอาจไม่สังเกตเห็นได้ชัดเจนเท่ากับรุ่น 8.0+

หมายเหตุ: หากคุณสงสัยว่า GPU ใดที่คุณควรใช้สำหรับการเรียนรู้เชิงลึก โปรดดูบล็อกโพสต์ของ Tim Dettmers " GPU ตัวใดสำหรับการเรียนรู้เชิงลึก" - https://timdettmers.com/2023/01/30/ซึ่ง-gpu-for-deep-learning/

* ในกรณีของเรา Rtx3050 ได้ 8.6 สามารถใช้ฟีเจอร์ pytorch 2.x ได้

In [7]:
# Make sure we're using a NVIDIA GPU
if torch.cuda.is_available():
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find("failed") >= 0:
    print("Not connected to a GPU, to leverage the best of PyTorch 2.0, you should connect to a GPU.")

  # Get GPU name
  gpu_name = !nvidia-smi --query-gpu=gpu_name --format=csv
  gpu_name = gpu_name[1]
  GPU_NAME = gpu_name.replace(" ", "_") # remove underscores for easier saving
  print(f'GPU name: {GPU_NAME}')

  # Get GPU capability score
  GPU_SCORE = torch.cuda.get_device_capability()
  print(f"GPU capability score: {GPU_SCORE}")
  if GPU_SCORE >= (8, 0):
    print(f"GPU score higher than or equal to (8, 0), PyTorch 2.x speedup features available.")
  else:
    print(f"GPU score lower than (8, 0), PyTorch 2.x speedup features will be limited (PyTorch 2.x speedups happen most on newer GPUs).")

  # Print GPU info
  print(f"GPU information:\n{gpu_info}")

else:
  print("PyTorch couldn't find a GPU, to leverage the best of PyTorch 2.0, you should connect to a GPU.")

GPU name: Tesla_T4
GPU capability score: (7, 5)
GPU score lower than (8, 0), PyTorch 2.x speedup features will be limited (PyTorch 2.x speedups happen most on newer GPUs).
GPU information:
Mon Dec  2 13:39:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              10W /  70W |      3MiB / 153

###1.1 ตั้งค่าอุปกรณ์ทั่วโลก
ก่อนหน้านี้ เราได้ตั้งค่าอุปกรณ์ของเทนเซอร์/รุ่นของเราโดยใช้ .to(device)

  * tensor.to (device)
  * model.to (device)

แต่ใน PyTorch 2.0 เป็นไปได้ที่จะตั้งค่าอุปกรณ์ด้วยตัวจัดการบริบทและอุปกรณ์ส่วนกลาง - https://pytorch.org/blog/pytorch-2.0-release/#beta-torchset_default_device-and-torchdevice-as-context -ผู้จัดการ

ดูเอกสาร - https://pytorch.org/tutorials/recipes/recipes/changing_default_device.html
* แทนที่เราจะใช้ .to(device) เราสามารถประการการตั้งค่า `torch.set_default_device(device) `เพื่อที่จะได้ไม่ต้องใช้`.to`แต่เปลี่ยนเป็น`.device`แทนได้ แต้ต้องประกาศ `torch.set_default_device("cpu or gpu") ` ก่อน

In [9]:
import torch

# Set the device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Set the device globally (requires pytorch 2.x+)
torch.set_default_device(device)

# All tensors or PyTorch objects created from here on out will be on the target device without using .to()
layer = torch.nn.Linear(20, 30)
print(f"Layer weights are on device: {layer.weight.device}")
print(f"Layer creating data on device: {layer(torch.randn(128, 20)).device}")

Layer weights are on device: cuda:0
Layer creating data on device: cuda:0


In [10]:
import torch

# Set the device globally (requires pytorch 2.x+)
torch.set_default_device("cpu")

# All tensors or PyTorch objects created from here on out will be on the target device without using .to()
layer = torch.nn.Linear(20, 30)
print(f"Layer weights are on device: {layer.weight.device}")
print(f"Layer creating data on device: {layer(torch.randn(128, 20)).device}")

Layer weights are on device: cpu
Layer creating data on device: cpu



##2. Setting up the experiments
Time to test speed!

To keep things simple, we'll run 4 experiments:

* Model: ResNet50 from torchvision
* Data: CIFAR10 from torchvision
Epochs: 5 (single run) and 3x5 (multi run)
* Batch size: 128 (note: you may want to change this depending on the amount of memory your GPU has, this tutorial focuses on an A100)
* Image size: 224 (note: you may want to adjust this given the amount of GPU memory you have)

In [11]:
import torch
import torchvision

print(f"PyTorch version: {torch.__version__}")
print(f"TorchVision version: {torchvision.__version__}")

# Set the target device
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using device: {device}")

PyTorch version: 2.5.1+cu121
TorchVision version: 0.20.1+cu121
Using device: cuda


###2.1 Create model and transforms


ResNet50 from PyTorch - https://pytorch.org/vision/stable/models/generated/torchvision.models.resnet50.html#torchvision.models.ResNet50_Weights
---



In [12]:
# Create model weights and transforms
model_weights = torchvision.models.ResNet50_Weights.IMAGENET1K_V2 # .DEFAULT also works here
transforms = model_weights.transforms()

>Note: PyTorch 2.0 relative speedups will be most noticeable when as much of the GPU as possible is being used. This means a larger model (more trainable parameters) may take longer to train on the whole but will relatively faster. E.g. a model with 1M parameters may take ~10 minutes to train but a model with 25M parameters may take ~20 minutes to train.

In [14]:
def create_model(num_classes=10):
  """
  Creates a resnet50 model with transforms and returns them both.
  """
  model_weights = torchvision.models.ResNet50_Weights.DEFAULT
  transforms = model_weights.transforms()
  model = torchvision.models.resnet50(weights=model_weights)

  # Adjust the head layer to suit our number of classes
  model.fc = torch.nn.Linear(in_features=2048,
                             out_features=num_classes)

  return model, transforms

model, transforms = create_model()
transforms

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 174MB/s]


ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

###2.2 การเร่งความเร็วจะสังเกตเห็นได้ชัดเจนที่สุดเมื่อมีการใช้ GPU ส่วนใหญ่
เนื่องจาก GPU สมัยใหม่ดำเนินการได้รวดเร็วมาก คุณจึงมักจะสังเกตเห็นการเร่งความเร็วส่วนใหญ่ที่สัมพันธ์กันเมื่อมีข้อมูลบน GPU มากที่สุดเท่าที่จะเป็นไปได้

ในทางปฏิบัติ โดยทั่วไปคุณต้องการใช้หน่วยความจำ GPU ของคุณให้ได้มากที่สุด

* การเพิ่มขนาดแบตช์ - เราใช้ขนาดแบตช์ 32 แต่สำหรับ GPU ที่มีความจุหน่วยความจำมากขึ้น โดยทั่วไปคุณจะต้องการใช้ให้ใหญ่ที่สุดเท่าที่จะเป็นไปได้ เช่น 128, 256, 512 ฯลฯ
* การเพิ่มขนาดข้อมูล - ตัวอย่างเช่น แทนที่จะใช้รูปภาพที่มีขนาด 32x32 ให้ใช้ 224x224 หรือ 336x336 นอกจากนี้คุณยังสามารถใช้ขนาดการฝังที่เพิ่มขึ้นสำหรับข้อมูลของคุณได้
* เพิ่มขนาดโมเดล - เช่น แทนที่จะใช้โมเดลที่มีพารามิเตอร์ 1M ให้ใช้โมเดลที่มีพารามิเตอร์ 10M
* การถ่ายโอนข้อมูลลดลง - เนื่องจากต้นทุนแบนด์วิธ (การถ่ายโอนข้อมูล) จะทำให้ GPU ช้าลง (เนื่องจากต้องการคำนวณข้อมูล)
จากการดำเนินการข้างต้น การเร่งความเร็วสัมพัทธ์ของคุณควรดีขึ้น

เช่น เวลาการฝึกโดยรวมอาจใช้เวลานานกว่าแต่ไม่เป็นเชิงเส้น

แหล่งข้อมูลสำหรับการเรียนรู้วิธีปรับปรุงความเร็วของโมเดล PyTorch - https://sebastianraschka.com/blog/2023/pytorch-faster.html

>หมายเหตุ: แนวคิดในการใช้ข้อมูลบน GPU ให้ได้มากที่สุดไม่ได้จำกัดเฉพาะ PyTorch 2.0 เท่านั้น แต่ยังใช้ได้กับทุกเวอร์ชันบน PyTorch และโดยทั่วไปคือทุกรุ่นที่ฝึกฝนบน GPU

###2.3 Checking the memory limits of our GPU
Can do so using torch.cuda - https://pytorch.org/docs/stable/generated/torch.cuda.mem_get_info.html#torch.cuda.mem_get_info

In [15]:
# Check available GPU memory and total GPU memory
total_free_gpu_memory, total_gpu_memory = torch.cuda.mem_get_info()
print(f"Total free GPU memory: {round(total_free_gpu_memory * 1e-9, 3)} GB")
print(f"Total GPU memory: {round(total_gpu_memory * 1e-9, 3)} GB")

Total free GPU memory: 15.692 GB
Total GPU memory: 15.836 GB


* If the GPU has 16GB+ of free memory, set batch size to 128
* If the GPU has less than 16GB of free, set batch size to 32

In [16]:
# Set batch size depending on amount of GPU memory
total_free_gpu_memory_gb = round(total_free_gpu_memory * 1e-9, 3)
if total_free_gpu_memory_gb >= 16:
  BATCH_SIZE = 128 # Note: you could experiment with higher values here if you like.
  IMAGE_SIZE = 224
  print(f"GPU memory available is {total_free_gpu_memory_gb} GB, using batch size of {BATCH_SIZE} and image size {IMAGE_SIZE}")
else:
  BATCH_SIZE = 32
  IMAGE_SIZE = 128
  print(f"GPU memory available is {total_free_gpu_memory_gb} GB, using batch size of {BATCH_SIZE} and image size {IMAGE_SIZE}")

GPU memory available is 15.692 GB, using batch size of 32 and image size 128


### 2.2 การเพิ่มความเร็วจะเห็นได้ชัดเจนที่สุดเมื่อใช้ GPU(s) เป็นส่วนใหญ่  

เนื่องจาก GPU สมัยใหม่มีความเร็วในการประมวลผลสูงมาก การเพิ่มความเร็วจะสังเกตได้ชัดเจนที่สุดเมื่อมีข้อมูลจำนวนมากถูกส่งไปให้ GPU ประมวลผล  

ในทางปฏิบัติ คุณควรใช้หน่วยความจำ GPU ให้เต็มประสิทธิภาพมากที่สุดเท่าที่จะทำได้  

- **เพิ่มขนาดแบตช์ (Batch size)**: เช่น หากปกติใช้ batch size 32 คุณอาจเพิ่มเป็น 128, 256 หรือ 512 หาก GPU มีหน่วยความจำมากพอ  
- **เพิ่มขนาดข้อมูล**: เช่น แทนที่จะใช้ภาพขนาด 32x32 อาจใช้ 224x224 หรือ 336x336 รวมถึงเพิ่มขนาด embedding ของข้อมูล  
- **เพิ่มขนาดโมเดล**: เช่น ใช้โมเดลที่มีพารามิเตอร์ 10 ล้านตัวแทนโมเดลที่มีเพียง 1 ล้านตัว  
- **ลดการถ่ายโอนข้อมูล (Data transfer)**: เนื่องจากการถ่ายโอนข้อมูลระหว่างหน่วยความจำและ GPU มีค่าใช้จ่ายด้านแบนด์วิดท์ที่อาจทำให้ GPU ทำงานช้าลง  

ด้วยการทำตามวิธีข้างต้น ความเร็วในการประมวลผลโดยรวมจะดีขึ้น แม้ว่าระยะเวลาการฝึกจะนานขึ้น แต่ไม่เพิ่มขึ้นแบบเชิงเส้น  

**แหล่งเรียนรู้เพิ่มเติม**: [วิธีทำให้โมเดล PyTorch เร็วขึ้น](https://sebastianraschka.com/blog/2023/pytorch-faster.html)  

หมายเหตุ: แนวคิดเรื่องการใช้ข้อมูลบน GPU ให้มากที่สุดนี้ไม่ได้จำกัดแค่ PyTorch 2.0 แต่ใช้ได้กับทุกเวอร์ชันของ PyTorch และโมเดลใด ๆ ก็ตามที่ฝึกบน GPU  

###2.3 Checking the memory limits of our GPU
Can do so using `torch.cuda` - https://pytorch.org/docs/stable/generated/torch.cuda.mem_get_info.html#torch.cuda.mem_get_info

In [17]:
# Check available GPU memory and total GPU memory
total_free_gpu_memory, total_gpu_memory = torch.cuda.mem_get_info()
print(f"Total free GPU memory: {round(total_free_gpu_memory * 1e-9, 3)} GB")
print(f"Total GPU memory: {round(total_gpu_memory * 1e-9, 3)} GB")


Total free GPU memory: 15.692 GB
Total GPU memory: 15.836 GB


* If the GPU has 16GB+ of free memory, set batch size to 128
* If the GPU has less than 16GB of free, set batch size to 32

In [18]:
# Set batch size depending on amount of GPU memory
total_free_gpu_memory_gb = round(total_free_gpu_memory * 1e-9, 3)
if total_free_gpu_memory_gb >= 16:
  BATCH_SIZE = 128 # Note: you could experiment with higher values here if you like.
  IMAGE_SIZE = 224
  print(f"GPU memory available is {total_free_gpu_memory_gb} GB, using batch size of {BATCH_SIZE} and image size {IMAGE_SIZE}")
else:
  BATCH_SIZE = 32
  IMAGE_SIZE = 128
  print(f"GPU memory available is {total_free_gpu_memory_gb} GB, using batch size of {BATCH_SIZE} and image size {IMAGE_SIZE}")


GPU memory available is 15.692 GB, using batch size of 32 and image size 128


In [19]:

transforms.crop_size = 128
transforms.resize_size = 128
print(f"Updated data transforms:\n{transforms}")

Updated data transforms:
ImageClassification(
    crop_size=128
    resize_size=128
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)


### 2.4 การเพิ่มความเร็วเพิ่มเติมด้วย TF32  

**TF32 (TensorFloat32)**  
TF32 เป็นชนิดข้อมูลที่อยู่กึ่งกลางระหว่าง **Float32** และ **Float16**  

- **Float32**: ตัวเลขถูกแทนด้วย 32 ไบต์ (เช่น 010101010110011 = 7)  
- **Float16**: ตัวเลขถูกแทนด้วย 16 ไบต์ (เช่น 01010101 = 4)  

**เป้าหมาย**:  
1. ทำให้การฝึกโมเดลเร็วขึ้น  
2. ทำให้การฝึกโมเดลมีความแม่นยำ  

TF32 เป็นชนิดข้อมูลที่พัฒนาโดย NVIDIA ซึ่งรวมข้อดีของ Float32 และ Float16 เข้าไว้ด้วยกัน  

TF32 ใช้งานได้กับ GPU สถาปัตยกรรม Ampere ขึ้นไป  

**เรียนรู้เพิ่มเติมเกี่ยวกับความแม่นยำ**: [Precision (Computer Science)](https://en.wikipedia.org/wiki/Precision_(computer_science))  


**Precision ในการคำนวณ**  
**Precision** คือความสามารถในการแทนค่าตัวเลขได้อย่างแม่นยำในหน่วยความจำหรือการคำนวณ  

- คิดเหมือนกับ **ท่อ**:  
  - **ท่อใหญ่ (Float32)**: เก็บน้ำ (ข้อมูล) ได้เยอะและละเอียด  
  - **ท่อเล็ก (Float16)**: เก็บน้ำได้น้อยกว่าและอาจไม่ละเอียดพอ  
  - **ท่อพอดี (TF32)**: เก็บน้ำได้เพียงพอและยังไหลเร็ว  

**ตัวอย่าง**:  
- Float32 = แม่นยำมากแต่ช้ากว่า  
- Float16 = เร็วแต่ไม่แม่นยำพอ  
- TF32 = สมดุลระหว่างความเร็วและความแม่นยำ  

**ผล**: Precision ส่งผลต่อความเร็วและคุณภาพของการประมวลผลโดยตรง!

In [20]:
GPU_SCORE

(7, 5)

In [21]:
if GPU_SCORE >= (8, 0): # check if GPU is compatiable with TF32
  print(f"[INFO] Using GPU with score: {GPU_SCORE}, enabling TensorFloat32")
  torch.backends.cuda.matmul.allow_tf32 = True
else:
  print(f"[INFO] Using GPU with score: {GPU_SCORE}, TensorFloat32 not available")
  torch.backends.cuda.matmul.allow_tf32 = False

[INFO] Using GPU with score: (7, 5), TensorFloat32 not available


###2.5 Preparing datasets
As before, we discussed we're going to use CIFAR10.

Homepage - https://www.cs.toronto.edu/~kriz/cifar.html

We can download the dataset from torchvision - https://pytorch.org/vision/main/generated/torchvision.datasets.CIFAR10.html

In [22]:
transforms

ImageClassification(
    crop_size=128
    resize_size=128
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [23]:
train_dataset = torchvision.datasets.CIFAR10(root=".",
                                             train=True,
                                             download=True,
                                             transform=transforms)
test_dataset = torchvision.datasets.CIFAR10(root=".",
                                             train=False,
                                             download=True,
                                             transform=transforms)
#Get len(dataset)
train_len = len(train_dataset)
test_len = len(test_dataset)
print(f"[INFO] Train dataset length: {train_len}")
print(f"[INFO] test dataset length: {test_len}")

100%|██████████| 170M/170M [00:03<00:00, 43.9MB/s]


Extracting ./cifar-10-python.tar.gz to .
Files already downloaded and verified
[INFO] Train dataset length: 50000
[INFO] test dataset length: 10000


In [24]:
train_dataset[0][1]

6

###2.6 Create Dataloaders

Next:
 * Turn datasets into DataLoaders

In [25]:
BATCH_SIZE * 2

64

In [26]:
from torch.utils.data import DataLoader

import os
NUM_WORKERS = os.cpu_count() # we want highest number of CPU cores to load data to GPU

train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=NUM_WORKERS)

test_dataloader = DataLoader(dataset=test_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=False,
                             num_workers=NUM_WORKERS)

# Print details:
print(f"Train dataloader num batches: {len(train_dataloader)} of batch size: {BATCH_SIZE}")
print(f"Test dataloader num batches: {len(test_dataloader)} of batch size: {BATCH_SIZE}")
print(f"Using num workers to load data (more is generally better): {NUM_WORKERS}")

Train dataloader num batches: 1563 of batch size: 32
Test dataloader num batches: 313 of batch size: 32
Using num workers to load data (more is generally better): 2


###2.7 Creating training and test loops
Want to create:

* Training and test loops + a timing step for each, so we know how long our models take to train/test
We covered this functionality in previous sections, one is here: https://www.learnpytorch.io/05_pytorch_going_modular/#4-creating-train_step-and-test_step-functions-and-train-to-combine-them

In [29]:
import time
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(epoch: int,
               model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device,
               disable_progress_bar: bool = False) -> Tuple[float, float]:
  """Trains a PyTorch model for a single epoch.

  Turns a target PyTorch model to training mode and then
  runs through all of the required training steps (forward
  pass, loss calculation, optimizer step).

  Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
  """
  # Put model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader data batches
  progress_bar = tqdm(
        enumerate(dataloader),
        desc=f"Training Epoch {epoch}",
        total=len(dataloader),
        disable=disable_progress_bar
    )

  for batch, (X, y) in progress_bar:
      # Send data to target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      y_pred = model(X)

      # 2. Calculate  and accumulate loss
      loss = loss_fn(y_pred, y)
      train_loss += loss.item()

      # 3. Optimizer zero grad
      optimizer.zero_grad()

      # 4. Loss backward
      loss.backward()

      # 5. Optimizer step
      optimizer.step()

      # Calculate and accumulate accuracy metrics across all batches
      y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
      train_acc += (y_pred_class == y).sum().item()/len(y_pred)

      # Update progress bar
      progress_bar.set_postfix(
            {
                "train_loss": train_loss / (batch + 1),
                "train_acc": train_acc / (batch + 1),
            }
        )


  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

def test_step(epoch: int,
              model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device,
              disable_progress_bar: bool = False) -> Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
  """
  # Put model in eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Loop through data loader data batches
  progress_bar = tqdm(
      enumerate(dataloader),
      desc=f"Testing Epoch {epoch}",
      total=len(dataloader),
      disable=disable_progress_bar
  )

  # Turn on inference context manager
  with torch.no_grad(): # no_grad() required for PyTorch 2.0, I found some errors with `torch.inference_mode()`, please let me know if this is not the case
      # Loop through DataLoader batches
      for batch, (X, y) in progress_bar:
          # Send data to target device
          X, y = X.to(device), y.to(device)

          # 1. Forward pass
          test_pred_logits = model(X)

          # 2. Calculate and accumulate loss
          loss = loss_fn(test_pred_logits, y)
          test_loss += loss.item()

          # Calculate and accumulate accuracy
          test_pred_labels = test_pred_logits.argmax(dim=1)
          test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

          # Update progress bar
          progress_bar.set_postfix(
              {
                  "test_loss": test_loss / (batch + 1),
                  "test_acc": test_acc / (batch + 1),
              }
          )

  # Adjust metrics to get average loss and accuracy per batch
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device,
          disable_progress_bar: bool = False) -> Dict[str, List]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    For example if training for epochs=2:
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]}
  """
  # Create empty results dictionary
  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": [],
      "train_epoch_time": [],
      "test_epoch_time": []
  }

  # Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs), disable=disable_progress_bar):

      # Perform training step and time it
      train_epoch_start_time = time.time()
      train_loss, train_acc = train_step(epoch=epoch,
                                        model=model,
                                        dataloader=train_dataloader,
                                        loss_fn=loss_fn,
                                        optimizer=optimizer,
                                        device=device,
                                        disable_progress_bar=disable_progress_bar)
      train_epoch_end_time = time.time()
      train_epoch_time = train_epoch_end_time - train_epoch_start_time

      # Perform testing step and time it
      test_epoch_start_time = time.time()
      test_loss, test_acc = test_step(epoch=epoch,
                                      model=model,
                                      dataloader=test_dataloader,
                                      loss_fn=loss_fn,
                                      device=device,
                                      disable_progress_bar=disable_progress_bar)
      test_epoch_end_time = time.time()
      test_epoch_time = test_epoch_end_time - test_epoch_start_time

      # Print out what's happening
      print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f} | "
          f"train_epoch_time: {train_epoch_time:.4f} | "
          f"test_epoch_time: {test_epoch_time:.4f}"
      )

      # Update results dictionary
      results["train_loss"].append(train_loss)
      results["train_acc"].append(train_acc)
      results["test_loss"].append(test_loss)
      results["test_acc"].append(test_acc)
      results["train_epoch_time"].append(train_epoch_time)
      results["test_epoch_time"].append(test_epoch_time)

  # Return the filled results at the end of the epochs
  return results

##3. Time models across a single run
Experiment 1: single run without torch.compile() for 5 epochs

3.1 Experiment 1 - Single run, no compile

In [27]:
NUM_EPOCHS = 5
LEARNING_RATE = 0.003

In [ ]:
model, _ = create_model() # _ จะเก็บค่า "extra_info" แต่ไม่ได้ใช้งานจริง
model.to(device)
# Create optimizer and loss function
optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=LEARNING_RATE)
loss_fn = torch.nn.CrossEntropyLoss()

# Train the classifier head of the pretrained resnet50 feature extractor model

single_run_no_compile_results = train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=NUM_EPOCHS,
                device=device)

  0%|          | 0/5 [00:00<?, ?it/s]

Training Epoch 0:   0%|          | 0/1563 [00:00<?, ?it/s]

Testing Epoch 0:   0%|          | 0/313 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.1172 | train_acc: 0.6018 | test_loss: 0.8404 | test_acc: 0.7019 | train_epoch_time: 189.4445 | test_epoch_time: 16.8857


Training Epoch 1:   0%|          | 0/1563 [00:00<?, ?it/s]

...

Epoch: 2 | train_loss: 0.6813 | train_acc: 0.7655 | test_loss: 0.6394 | test_acc: 0.7832 | train_epoch_time: 188.2464 | test_epoch_time: 16.2986


Training Epoch 2:   0%|          | 0/1563 [00:00<?, ?it/s]

Testing Epoch 2:   0%|          | 0/313 [00:00<?, ?it/s]

Epoch: 3 | train_loss: 0.5221 | train_acc: 0.8203 | test_loss: 0.5637 | test_acc: 0.8080 | train_epoch_time: 192.8414 | test_epoch_time: 16.3168


Training Epoch 3:   0%|          | 0/1563 [00:00<?, ?it/s]

##3.2 Experiment 2, single, using torch.compile()
Same setup as experiment 1 except with the new line `torch.compile().`

In [ ]:
model, _ = create_model() # _ จะเก็บค่า "extra_info" แต่ไม่ได้ใช้งานจริง
model.to(device)
# Create optimizer and loss function
optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=LEARNING_RATE)
loss_fn = torch.nn.CrossEntropyLoss()

compile_start_time = time.time()
###new in pytorch 2.x ###
compile_model = torch.compile(model)

compile_end_time = time.time(_)

compile_time = compile_end_time - compile_start_time

print(f"Time to compile: {compile_time} |\nNote: the first time you compile a model/train a compiled model, the first epoch may take longer due to optimizations happening behind the scenes")

# Train the compiled model

single_run_no_compile_results = train(model=compile_model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=NUM_EPOCHS,
                device=device)

###3.3 Compare the results of experiment 1 and 2
Nice!

We've got two trained models:

1. One without torch.compile().
2. One with torch.compile().
Let's compare the results of each experiment.

To do so, we'll first create dataframes of the results of each.

Then we'll plot the results of each experiment on a bar chart.

In [ ]:
import pandas as pd
single_run_no_compile_results_df = pd.DataFrame(single_run_no_compile_results)
single_run_compile_results_df = pd.DataFrame(single_run_compile_results)

In [ ]:
# Check out the head of one of the results dataframes
single_run_no_compile_results_df.head()

Got the results for experiments 1 and 2!

Now let's write a function to take in the results and compare them with a bar chart.

We'll add some metadata to the function so it can display some information about the experiments.

Namely all of the parameters in our experiment setup:

  * The dataset name.
  * The model name.
  * The number of epochs.
  * The batch size.
  * The image size.

###3.4 Save single run results to file with GPU details

In [ ]:
# Make a directory for single_run results
import os
pytorch_2_results_dir = "pytorch_2_results"
pytorch_2_single_run_results_dir = f"{pytorch_2_results_dir}/single_run_results"
os.makedirs(pytorch_2_single_run_results_dir, exist_ok=True)

# Create filenames for each of the dataframes
save_name_for_non_compiled_results = f"single_run_non_compiled_results_{DATASET_NAME}_{MODEL_NAME}_{GPU_NAME}.csv"
save_name_for_compiled_results = f"single_run_compiled_results_{DATASET_NAME}_{MODEL_NAME}_{GPU_NAME}.csv"

# Create filepaths to save the results to
single_run_no_compile_save_path = f"{pytorch_2_single_run_results_dir}/{save_name_for_non_compiled_results}"
single_run_compile_save_path = f"{pytorch_2_single_run_results_dir}/{save_name_for_compiled_results}"
print(f"[INFO] Saving non-compiled experiment 1 results to: {single_run_no_compile_save_path}")
print(f"[INFO] Saving compiled experiment 2 results to: {single_run_compile_save_path}")

# Save the results
single_run_no_compile_results_df.to_csv(single_run_no_compile_save_path)
single_run_compile_results_df.to_csv(single_run_compile_save_path)

##4. Time models across multiple runs
Time for multi-run experiments!

  * Experiment 3 - 3x5 epochs without torch.compile()
  * Experiment 4 - 3x5 epochs with torch.compile()
Before running experiment 3 and 4, let's create 3 functions:

  1. Experiment 3: create_and_train_non_compiled_model() - creates and trains a model for a single run (can put this function in a loop for mulitple runs).
  2. Experiment 4: create_compiled_model() - creates and compiles a model, returns the compiled model.
  3. Experiment 4: train_compiled_model() - trains a compiled model for a single run (can put this in a loop to train for multiple runs).

**ความหมายของผลลัพธ์:**  

1. **Mean train epoch time difference: -10.932%**  
   - เวลาเฉลี่ยในการฝึก (train) ของแต่ละ epoch ลดลง **10.932%** เมื่อเปรียบเทียบระหว่างโมเดลที่ใช้ **PyTorch 2.0 (compiled)** กับโมเดลแบบธรรมดา (non-compiled)  
   - **ค่าลบ (negative)** หมายถึงโมเดลที่ถูก compile เร็วกว่า  

2. **Mean test epoch time difference: 3.373%**  
   - เวลาเฉลี่ยในการทดสอบ (test) ของแต่ละ epoch เพิ่มขึ้น **3.373%** เมื่อใช้โมเดลที่ compile  
   - **ค่าบวก (positive)** หมายถึงการทดสอบ (test) ของโมเดลแบบ compile ใช้เวลานานกว่าเล็กน้อย  

---

### **สรุปผลโดยรวม:**  
- **การฝึก (train)**: PyTorch 2.0 (compiled) ช่วยให้กระบวนการฝึกเร็วขึ้นประมาณ 10.932%  
- **การทดสอบ (test)**: PyTorch 2.0 (compiled) อาจมี overhead เพิ่มขึ้นเล็กน้อยในขั้นตอนการทดสอบ  

---

### **สาเหตุที่การทดสอบช้าลง:**  
1. **Overhead จากการ Optimize Model:**  
   การ compile โมเดลอาจเพิ่มขั้นตอนในระหว่างการรัน (เช่น การ optimize graph)  
2. **ขนาดข้อมูลใน Test:**  
   หากข้อมูลในขั้นตอนการทดสอบมีขนาดเล็ก อาจไม่ได้ใช้ประสิทธิภาพของการ compile เต็มที่  

---

### **ควรเลือกใช้ PyTorch 2.0 หรือไม่?**  
- **เหมาะสมถ้า:**  
   - งานของคุณเน้นการฝึกโมเดล (training) บ่อยครั้ง  
   - ต้องการประหยัดเวลาในการรันโมเดลขนาดใหญ่  
- **ควรระวังถ้า:**  
   - การทดสอบโมเดลใช้เวลาสำคัญมาก เช่น งานที่ต้องการ inference แบบ real-time  



###5. Possible improvements and extensions
  1. More powerful CPUs - to load data to the GPU as quickly as possible, relative speedups come from using as much of the GPU as possible.
  2.Use mixed precision training - uses a combination of float32 and float16 to train a model (generally improves model training time).
  3.Transformer based models may see more relative speedups than convolutional -
  4.Train for longer - the longer we train, the more we can benefit from the optimization steps that torch.compile() does up front.
  See more here: https://www.learnpytorch.io/pytorch_2_intro/#5-possible-improvements-and-extensions